## Hybrid Algorithm

In [ ]:
In attempt to join the qualities of motion detection algorithms a hybrid algorithm was proposed,
which is a modification of 3-frame difference with background adaptation problems. The implemented
algorithm is presented in the figure below. The output with analysis against different conditions will help us
classify the optimum algorithm for our system.


<img s>